import requests
from bs4 import BeautifulSoup
import re

url = "https://pokemondb.net/go/evolution"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

family_map = {}

# Each evolution family is inside a div with class 'infocard-list-evo'
for evo_chain in soup.select("div.infocard-list-evo"):
    # Find the base form dex number from the first sprite alt attribute
    base_card = evo_chain.select_one("span.infocard > a.ent-name")
    base_dex = None
    if base_card and base_card["href"]:
        sprite = evo_chain.select_one("span.infocard img")
        if sprite and "alt" in sprite.attrs:
            alt_text = sprite["alt"]  # e.g., "#001 Bulbasaur"
            match = re.match(r"#(\d+)", alt_text)
            if match:
                base_dex = int(match.group(1))

    if base_dex is None:
        # Could not find base dex number, skip this family
        continue

    # Convert the whole evo_chain HTML to a string
    evo_html_str = str(evo_chain)

    # Find all occurrences of # followed by up to 5 digits
    dex_numbers = set()
    for match in re.finditer(r"#(\d{1,5})", evo_html_str):
        dex_num = int(match.group(1))
        dex_numbers.add(dex_num)

    # Store in map: base_dex -> sorted list of all family dex numbers
    family_map[base_dex] = sorted(dex_numbers)

# Output the family map as JavaScript object
print("const evolutionFamilies = {")
for base, dex_list in sorted(family_map.items()):
    dex_str = ", ".join(str(d) for d in dex_list)
    print(f"  {base}: [{dex_str}],")
print("};")
